## Modularising the Visualisations

In [7]:
import pandas as pd
from base64 import b64encode
import folium
import branca
import json
from branca.element import MacroElement
from branca.colormap import linear, LinearColormap
from jinja2 import Template
from folium import plugins
import matplotlib.pyplot as plt

In [8]:
class BindColormap(MacroElement):
    def __init__(self, layer, colormap):
        super(BindColormap, self).__init__()
        self.layer = layer
        self.colormap = colormap
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
            {{this._parent.get_name()}}.on('overlayadd', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
                }});
            {{this._parent.get_name()}}.on('overlayremove', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'none';
                }});
        {% endmacro %}
        """)

In [21]:
class baseMap():
    """Creates a base map to create and plot visualisations
    
    Parameters
    ----------
    data : pandas dataframe or path to CSV file
        Dataset that would be used for creating the map.
        
    shapefile : path to a GeoJSON file
        Shapefile for plotting on a map    
    
    Examples
    --------
    >>> map.groupedLayerControl()
    
    """
    def __init__(self, data = 'None', shapefile = None):
        self.map = folium.Map(location = [2,-2], zoom_start = 3)
        self.fg_geo = {}
        self.fgfacts = {}
        self.fgspark = {}
        if type(data) == str:
            if data == 'None':
                self.df = pd.DataFrame(columns=['baseMap'])
            elif data[-4:] == '.csv':
                self.df = pd.read_csv(data)
            else:
                print('Enter path to a csv file')
        else:
            self.df = data
        if shapefile == None:
            pass
        elif shapefile[-8:] == '.geojson':
            self.geodata = json.load(open(shapefile))
        else:
            print("Enter a GeoJson file.")
            
            
    def groupedLayerControl(self, radio = []):
        
        """Creates a grouped layer control.
        
        Parameters
        --------
        radio : List of groups that should have mutually exclusive items.
    
        Examples
        --------
        >>> map = baseMap()
        >>> map = baseMap(data = 'data/alldata.csv', shapefile = 'data/country3.0.geojson' )

        """
        if (bool(self.fg_geo) == True) & ('Location Hierarchy' in radio):
            self.fg_geo['None'] = folium.map.FeatureGroup(name='None', show=False).add_to(self.map)
        if (bool(self.fgfacts) == True) & ('Facts' in radio):
            self.fg_geo['None'] = folium.map.FeatureGroup(name='None', show=False).add_to(self.map)
        if (bool(self.fgspark) == True) & ('Sparkline' in radio):
            self.fg_geo['None'] = folium.map.FeatureGroup(name='None', show=False).add_to(self.map)
        folium.plugins.GroupedLayerControl({}, 
                                    {'Location Hierarchy' : self.fg_geo,
                                    'Facts': self.fgfacts, 
                                    'Sparkline': self.fgspark }, 
                                  ['Location Hierarchy']).add_to(self.map)
        
    def getMap(self):
        """Returns a folium map object.
        
        Returns
        --------
        folium map object

        Examples
        --------
        >>> bmap.getMap()

        """
        return self.map
    
    

class geojson():
    
    """Creates a Choropleth map.
    
    Parameters
    ----------
    baseMap : A baseMap object is given to the function
    
    name : Name of the Choropleth map being created
        
    data : pandas dataframe or path to CSV file
        Dataset that would be used for creating the map.
       
    shapefile : path to a GeoJSON file
        Shapefile for plotting on a map    

    Returns
    -------
    geojson object
    
    Examples
    --------
    >>> Geo = geojson(basemap, 'Choropleth', shapefile = 'geodata/city3.0.geojson')
    
    """
    def __init__(self, baseMap, name, data = 'None', shapefile = None):
        self.baseMap = baseMap
        self.name = name
        self.vdict, self.vdict2 = {}, {}
        self.column1, self.column2 = None, None
        self.colormap = None
        self.colormap2 = None
        self.plotcolumns, self.plotlabels = [],[]
        self.valuecolumns, self.valuestring = [], []
        if type(data)==str:
            if data == 'None':
                self.df=self.baseMap.df
            elif data[-4:0]=='.csv':
                self.df = pd.read_csv('data')
            else: 
                print("Enter path to a csv file.")
        else:
            self.df = data
        if shapefile == None:
            self.geodata = self.baseMap.geodata
        else:
            self.geodata = json.load(open(shapefile))
        self.baseMap.fg_geo[name] = folium.map.FeatureGroup(name=name, show=False).add_to(self.baseMap.map)
        

    def colorMap(self, column1 = None, column2 = None):
        
        """Creates a colormap for the Choropleth map.

        Parameters
        ----------
        column1 : Attribute that specifies the fill colour of the Choropleth map

        column2 : Attribute that spceifies the border colour of the Choropleth map

        Examples
        --------
        >>> geo.colorMap('Population', 'PopDensity')

        """
        
        if column1 == None:
            pass
        else:
            self.column1 = column1
            self.colormap = LinearColormap(['#3f372f', '#62c1a6', '#1b425e','#bbf9d9', '#ffffff'],vmin=self.df[column1].min(),vmax=self.df[column1].max()).to_step(data=self.df[column1], n=5, method='quantiles')
            self.vdict = self.df.set_index('Location')[column1]
            self.baseMap.map.add_child(self.colormap)        
            self.baseMap.map.add_child(BindColormap(self.baseMap.fg_geo[self.name], self.colormap))
        if column2 == None:
            self.colormap2 = None
        else:
            self.column2 = column2
            self.colormap2=LinearColormap(['#3f372f', '#62c1a6', '#1b425e','#bbf9d9', '#ffffff'],vmin=self.df[column2].min(),vmax=self.df[column2].max()).to_step(data=self.df[column2], n=5, method='quantiles')
            self.vdict2 = self.df.set_index('Location')[column2]
            self.baseMap.map.add_child(self.colormap2)
            self.baseMap.map.add_child(BindColormap(self.baseMap.fg_geo[self.name], self.colormap2))

    def createMap(self):
        
        """Creates the Choropleth map.

        Examples
        --------
        >>> geo.createMap()

        """
        
        folium.GeoJson(self.geodata, 
                       style_function=lambda feature: {
        'fillColor': (self.colormap(self.vdict[feature['properties']['name']]) if feature['properties']['name'] in self.vdict else 'grey') if ~(self.column1 == None) else 'black',
      'color': (self.colormap2(self.vdict2[feature['properties']['name']]) if feature['properties']['name'] in self.vdict2 else 'grey') if ~(self.column2 == None) else 'black',
      'weight': 2 if (self.column2 != None) else 0.5,
      'fillOpacity': 1 if feature['properties']['name'] in self.vdict else 0},
       tooltip=folium.features.GeoJsonTooltip(fields=['name'],
                                          labels=False,
                                          sticky=False)).add_to(self.baseMap.fg_geo[self.name])
        
        
    def addValue(self, columns, string):
        
        """Takes input for the information to be displayed in the information box as text.
    
        Parameters
        ----------
        columns : The column value to be displayed

        string : The string following the value 

        Examples
        --------
        >>> geo.addValue("ven/pop", " of the population are Venezuelans.")
        >>> geo.addValue("ven/migrants", " of the migrants are Venezuelans.")

            """
        self.valuecolumns.append(columns)
        self.valuestring.append(string)
        
    def createPlots(self, columns, labels):
        
        """Takes input for the plots to be displayed in the information box.
    
        Parameters
        ----------
        columns : A list of column names used to create the pie chart.

        labels: A list of labels corresponding to the column values for the pie chart.

        Examples
        --------
        >>> geo.addValue("ven/pop", " of the population are Venezuelans.")
        >>> geo.addValue("ven/migrants", " of the migrants are Venezuelans.")

        """
        
        self.plotcolumns.append(columns)
        self.plotlabels.append(labels)
    
    def copyInfoBox(self, geojson):
        
        """Copies the structure of information box created for one geojson object to another.
    
        Parameters
        ----------
        geojson : A geojson object whose information box's structure is to be copied.

        Examples
        --------
        >>> geo.copyInfoBox(geojson1)

        """
        self.valuecolumns = geojson.valuecolumns
        self.valuestring = geojson.valuestring
        self.plotcolumns = geojson.plotcolumns
        self.plotlabels = geojson.plotlabels
        
    
    def addInfoBox(self):
        
        """Adds information box to the choropleth map as a pop up.
        
        Examples
        --------
        >>> geo.addInfoBox()
        """
        
        for feature in self.geodata['features']:
            a = self.df[self.df["Location"] == feature['properties']['name']]
            ind=(a.index)[0]
            html = "<center><h2 style='font-family: Arial, Helvetica, sans-serif;'>" + feature['properties']['name'] + "</h2></center><center>"
            for i in range(len(self.valuecolumns)):
                html+="<h4 style='font-family: Arial, Helvetica, sans-serif;'>"+str(round(a.loc[ind][self.valuecolumns[i]]*100, 2))+"%" + self.valuestring[i] + "</h4>"
            encodedlist = []
            for i in range(len(self.plotcolumns)):
                fig1, ax1 = plt.subplots(figsize=(5,3))
                ax1.pie([a.loc[(a.index)[0],j] for j in self.plotcolumns[i]], labels=self.plotlabels[i], autopct='%1.1f%%', shadow=True, startangle=90)
                plt.savefig('myfig.png', transparent = True)
                encoded = b64encode(open('myfig.png', 'rb').read()).decode()
                encodedlist.append(encoded)
                html += '<img align="middle" src="data:image/png;base64,{}">'

            html += '</center>'

            geo = folium.GeoJson(feature['geometry'],
                   style_function=lambda feature: { 'weight': 0,'fillOpacity': 0},
                    tooltip =feature['properties']['name'])
            
            iframe = branca.element.IFrame(html=html.format(*encodedlist), width=400, height=400)
            folium.Popup(iframe).add_to(geo)  
            geo.add_to(self.baseMap.fg_geo[self.name])
            
class interestingFacts():
    """Creates a map with icons showing interesting facts
    
    Parameters
    ----------
    baseMap : A baseMap object is given to the function
    
    name : Name of the feature group being created
        
    data : pandas dataframe or path to CSV file
        Dataset that would be used for creating the map.  
    
    Examples
    --------
    >>> f= interestingFacts(basemap, 'City', data = df)
    
    """
    def __init__(self, basemap, name, data = 'None'):
        self.basemap = basemap
        self.name = name
        self.Loc= []
        self.tdict={}
        self.pdict={}
        self.pidict={}
        self.hdict={}
        self.cdict={}
        if type(data)==str:
            if data == 'None':
                self.df=self.basemap.df
            elif data[-4:0]=='.csv':
                self.df = pd.read_csv('data')
            else: 
                print("Enter path to a csv file.")
        else:
            self.df = data
        self.basemap.fgfacts[self.name] = folium.map.FeatureGroup(name=self.name, show=True).add_to(self.basemap.map)
            
    
    def addFacts(self, array, labels, icon_map, icon_pop):
        """Creates icons on map

        Parameters
        ----------
        array : List of column name
        label : Tuple of label name 
        icon_map : the icon shown on map
        icon_pop : pop up icon

        Examples
        --------
        >>> f.addFacts(['%ven_audience_woman', '%ven_audience_man'], ('Women', 'Men'), '1.png','2.png')

        """
        df=self.df
        for i in range(len(array)):
            arr = [0]*len(array)
            a = df.loc[df[array[i]].idxmax(axis=1)]
            for j in range(len(array)):
                arr[j]=a[array[j]]
            fig1, ax1 = plt.subplots(figsize=(1.8,1.8))
            ax1.pie(arr, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90)
            plt.savefig('myfig.png', transparent = True)
            html1 = '<center><img align="middle" src="data:image/png;base64,{}"></center>'
            encoded = b64encode(open(icon_pop, 'rb').read()).decode()
            encpie = b64encode(open('myfig.png', 'rb').read()).decode()
            ic = icon_map
            l = a.Location
            if l not in self.Loc:
                self.Loc.append(l)
                self.pdict[l], self.hdict[l],self.tdict[l], self.pidict[l], self.cdict[l] = '<h4><center>' + l +'</center></h4>', '', [], [], 0
            self.pdict[l] += '<hr><center><p style="padding:0px 10px 0px 10px">'+ l
            self.hdict[l] += html1
            self.tdict[l].append(encoded)
            self.pidict[l].append(encpie)
            self.cdict[l]+=1
            string = " highest percentage of "+ labels[i]+"."
            self.pdict[l] += ' has the'+string+'</p></center><center><img align="middle" src="data:image/png;base64,{}"></center>' 
            iframe = branca.element.IFrame(html=self.pdict[l].format(*self.pidict[l]), width=400, height = 220 + self.cdict[l]* 50)
            folium.Marker([a.LatLong.split(",")[0], a.LatLong.split(",")[1]], popup = folium.Popup(iframe), icon = folium.features.CustomIcon(ic,icon_size=(28, 30)),tooltip=self.hdict[l].format(*self.tdict[l]) ).add_to(self.basemap.fgfacts[self.name])


In [ ]:
df3 = pd.read_csv('data.csv')
df3 = df3[df3["Frequency"] == "Monthly"][df3["LocationHierarchy"] == 'City']
df2 = pd.read_csv('data.csv')
df2 = df2[df2["Frequency"] == "Monthly"][df2["LocationHierarchy"] == 'Country']
df4 = pd.read_csv('data.csv')
df4 = df4[df4["Frequency"] == "Monthly"][df4["LocationHierarchy"] == 'State']


bmap = baseMap(data = 'data.csv')

#City
g = geojson(bmap, 'City', data = df3, shapefile = 'geodata/city3.0.geojson')
g.colorMap('venezuelans', 'ven/pop')
g.createMap()

g.addValue("ven/pop", " of the population are Venezuelans.")
g.addValue("ven/migrants", " of the migrants are Venezuelans.")
g.createPlots(["%ven_audience_man", "%ven_audience_woman"], ['Men', 'Women'])
g.createPlots(["%ven_audience_iOS", "%ven_audience_Android", "%ven_audience_Other"], ["iOS", "Android", "Others"])
g.createPlots(["%ven_audience_graduated", "%ven_audience_high_school", "%ven_audience_no_degree"], ["Graduated", "High School", "No degree"])
g.createPlots(["%ven_audience_single", "%ven_audience_dating", "%ven_audience_married"], ["Single", "Dating", "Married"])
g.createPlots(["%ven_audience_adolecent", "%ven_audience_young_adult", "%ven_audience_adult", "%ven_audience_middle_age", "%ven_audience_elder"], ["adolescent", "young adult", "adult", "middle age", "elder"])

g.addInfoBox()

#Country

g2 = geojson(bmap, 'Country', data = df2, shapefile = 'geodata/country3.0.geojson')
g2.colorMap('venezuelans')
g2.createMap()
g2.copyInfoBox(g)
g2.addInfoBox()

#States

g3 = geojson(bmap, 'State', data = df4, shapefile = 'geodata/state3.0.geojson')
g3.colorMap('venezuelans')
g3.createMap()
g3.copyInfoBox(g)
g3.addInfoBox()



f= interestingFacts(bmap, 'City', data = df3)
f.addFacts(['%ven_audience_woman', '%ven_audience_man'], ('Women', 'Men'), 'colab/venflag2.png','colab/iconr1.png')
f.addFacts(['%ven_audience_iOS', '%ven_audience_Android', '%ven_audience_Other'], ('iOS', 'Android', 'Other'), 'colab/venflag2.png', 'iconr2.png')
f.addFacts(['%ven_audience_graduated', '%ven_audience_high_school','%ven_audience_no_degree'], ('Graduated', 'High School', 'No degree'), 'colab/venflag2.png', 'iconr3.png')
f.addFacts(['%ven_audience_single', '%ven_audience_dating', '%ven_audience_married'], ('Single', 'Dating', 'Married'), 'colab/venflag2.png', 'iconr4.png')


bmap.groupedLayerControl()
m = bmap.getMap()
m.save('June17.html')